# Import Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### First Look on Dataset

In [ ]:
data=pd.read_csv(r'/kaggle/input/twitter-user-gender-classification/gender-classifier-DFE-791531.csv',encoding='latin1')

In [ ]:
data.head()

### Data Preprocessing

In [ ]:
data=pd.concat([data.gender,data.description],axis=1)
data.dropna(axis=0,inplace=True)
data.head()


In [ ]:
data.gender=[1 if k=='female' else 0 for k in data.gender]
data.gender.value_counts()

### Regular Expression

In [ ]:
import re

first=data.description[4]
description=re.sub("[^a-zA-Z]"," ",first)
description=description.lower()
description

### Stopwords

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
description=nltk.word_tokenize(description)
description

In [ ]:
description=[word for word in description if not word in set(stopwords.words('english'))]

### Lemmatization

In [ ]:
import nltk as nlp

lemma=nlp.WordNetLemmatizer()
description=[lemma.lemmatize(word) for word in description]
description=" ".join(description)
description

### Data Cleaning

In [ ]:
description_list=[]
for description in data.description:
    description=re.sub("[^a-zA-Z]"," ",description)
    description=description.lower()
    description=nltk.word_tokenize(description)
    #description=[word for word in description if not word in set(stopwords.words('english'))]
    description=[lemma.lemmatize(word) for word in description]
    description=" ".join(description)
    description_list.append(description)
    

### Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
max_features=5000#kelimelerin içinden en cok kullanılanları seç
count_vectorizer=CountVectorizer(max_features=max_features,stop_words='english')
sparce_matrix=count_vectorizer.fit_transform(description_list).toarray()
print('en sık {} kelime: {}'.format(max_features,count_vectorizer.get_feature_names()))

### Text Classification

In [ ]:
y=data.iloc[:,0].values#male or female classes
x=sparce_matrix
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)
#with naive bayes
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(x_train,y_train)
#prediction
y_pred=nb.predict(x_test)
print('accuracy score : ',nb.score(y_pred.reshape(-1,1),y_test))